In [1]:
from urllib.request import urlretrieve
import os
import numpy as np
import tensorflow as tf
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cv2

def divide_img():
    ra_dataframe  = pd.read_csv('./rough_data.csv', header=None)
    files         = glob.glob('./right/*')
    files.extend(glob.glob('./left/*'))
    process_dataframe = pd.DataFrame(columns=['path','value','class'])
    print(files)
    for file_path in files:
        left,right = file_path.split("/", 1)
        dire,right = right.split("/", 1)
        num,right  = right.split("_",1)

        row        = 0
        col        = 0
        if dire == 'left':
            col = 0
        else:
            col = 1
        row = int(num) - 1
        ra_value = ra_dataframe.iloc[row,col]
        if float(ra_value) < 1.5:
            os.makedirs('./rough1.0-1.5_nod', exist_ok=True)
            #divide image to six images
            image = cv2.imread(file_path)
            
            store_path = './rough1.0-1.5_nod/' + str(row+1) + '_' + dire + '_1' + '.jpg'
            cv2.imwrite(store_path, image)

        elif float(ra_value) >= 1.5 and float(ra_value) < 2.0:
            os.makedirs('./rough1.5-2.0_nod/', exist_ok=True)
            image = cv2.imread(file_path)
            store_path = './rough1.5-2.0_nod/' + str(row+1) + '_' + dire + '_1' + '.jpg'
            cv2.imwrite(store_path, image)

        elif float(ra_value) >= 2.0:
            os.makedirs('./rough2.0-2.5_nod/', exist_ok=True)
            image = cv2.imread(file_path)
            store_path = './rough2.0-2.5_nod/' + str(row+1) + '_' + dire + '_1' + '.jpg'
            cv2.imwrite(store_path, image)
        
def load_img(path):
    img = skimage.io.imread(path)
    img = img / 255.0
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    resized_img = skimage.transform.resize(crop_img, (224, 224))[None, :, :, :]   # shape [1, 224, 224, 3]
    return resized_img

def load_data():
    imgs   = {'rough1.0-1.5_nod': [] , 'rough1.5-2.0_nod': [],'rough2.0-2.5_nod': []}
    answer = {'rough1.0-1.5_nod': [] , 'rough1.5-2.0_nod': [], 'rough2.0-2.5_nod': []}
    #format 6_right_1
    for k in imgs.keys():
        dir = './' + k + '/'
        for file in os.listdir(dir):
            if not file.lower().endswith('.jpg'):
                continue
            try:
                resized_img = load_img(os.path.join(dir, file))
            except OSError:
                continue
            imgs[k].append(resized_img)
            if k == 'rough1.0-1.5_nod':
                answer[k].append(0.0)
            elif k == 'rough1.5-2.0_nod':
                answer[k].append(1.0)
            elif k == 'rough2.0-2.5_nod':
                answer[k].append(2.0)
    
    return imgs['rough1.0-1.5_nod'], imgs['rough1.5-2.0_nod'], imgs['rough2.0-2.5_nod'],answer['rough1.0-1.5_nod'],answer['rough1.5-2.0_nod'],answer['rough2.0-2.5_nod']

class Vgg16:
    vgg_mean = [103.939, 116.779, 123.68]

    def __init__(self, vgg16_npy_path=None, restore_from=None):
        # pre-trained parameters
        try:
            self.data_dict = np.load(vgg16_npy_path, encoding='latin1').item()
        except FileNotFoundError:
            print('Please download VGG16 parameters from here https://mega.nz/#!YU1FWJrA!O1ywiCS2IiOlUCtCpI6HTJOMrneN-Qdv3ywQP5poecM\nOr from my Baidu Cloud: https://pan.baidu.com/s/1Spps1Wy0bvrQHH2IMkRfpg')

        self.tfx = tf.placeholder(tf.float32, [None, 224, 224, 3])
        self.tfy = tf.placeholder(tf.float32, [None, 1])

        # Convert RGB to BGR
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=self.tfx * 255.0)
        bgr = tf.concat(axis=3, values=[
            blue - self.vgg_mean[0],
            green - self.vgg_mean[1],
            red - self.vgg_mean[2],
        ])

        # pre-trained VGG layers are fixed in fine-tune
        conv1_1 = self.conv_layer(bgr, "conv1_1")
        conv1_2 = self.conv_layer(conv1_1, "conv1_2")
        pool1 = self.max_pool(conv1_2, 'pool1')

        conv2_1 = self.conv_layer(pool1, "conv2_1")
        conv2_2 = self.conv_layer(conv2_1, "conv2_2")
        pool2 = self.max_pool(conv2_2, 'pool2')

        conv3_1 = self.conv_layer(pool2, "conv3_1")
        conv3_2 = self.conv_layer(conv3_1, "conv3_2")
        conv3_3 = self.conv_layer(conv3_2, "conv3_3")
        pool3   = self.max_pool(conv3_3, 'pool3')

        conv4_1 = self.conv_layer(pool3, "conv4_1")
        conv4_2 = self.conv_layer(conv4_1, "conv4_2")
        conv4_3 = self.conv_layer(conv4_2, "conv4_3")
        pool4   = self.max_pool(conv4_3, 'pool4')

        conv5_1 = self.conv_layer(pool4, "conv5_1")
        conv5_2 = self.conv_layer(conv5_1, "conv5_2")
        conv5_3 = self.conv_layer(conv5_2, "conv5_3")
        pool5 = self.max_pool(conv5_3, 'pool5')

        # detach original VGG fc layers and
        # reconstruct your own fc layers serve for your own purpose
        self.flatten = tf.reshape(pool5, [-1, 7*7*512])
        
        self.fc6 = tf.layers.dense(self.flatten, 256, tf.nn.relu, name='fc6')
        self.out = tf.layers.dense(self.fc6, 1, name='out')

        self.sess = tf.Session()
        if restore_from:
            saver = tf.train.Saver()
            saver.restore(self.sess, restore_from)
        else:   # training graph
            self.loss = tf.losses.mean_squared_error(labels=self.tfy, predictions=self.out)
            self.train_op = tf.train.RMSPropOptimizer(0.001).minimize(self.loss)
            self.sess.run(tf.global_variables_initializer())

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def conv_layer(self, bottom, name):
        with tf.variable_scope(name):   # CNN's filter is constant, NOT Variable that can be trained
            conv = tf.nn.conv2d(bottom, self.data_dict[name][0], [1, 1, 1, 1], padding='SAME')
            lout = tf.nn.relu(tf.nn.bias_add(conv, self.data_dict[name][1]))
            return lout

    def train(self, x, y):
        loss, _ = self.sess.run([self.loss, self.train_op], {self.tfx: x, self.tfy: y})
        return loss

    def predict(self, paths):
        fig, axs = plt.subplots(1, 2)
        imgs   = {'test_nod': [] }
        for k in imgs.keys():
            dir = './' + k + '/'
            for file in os.listdir(dir):
                if not file.lower().endswith('.jpg'):
                    continue
                resized_img = load_img(os.path.join(dir, file))
                length = self.sess.run(self.out, {self.tfx: resized_img})
                print(file)
                print(length)
        
    def save(self, path='./for_transfer_learning/model/transfer_learn'):
        saver = tf.train.Saver()
        saver.save(self.sess, path, write_meta_graph=False)

def train():
    rough0_x, rough1_x, rough2_x , rough0_y, rough1_y, rough2_y = load_data()
    # plot fake length distribution
    rough0_y = np.resize(np.array(rough0_y),(len(rough0_y),1))
    rough1_y = np.resize(np.array(rough1_y),(len(rough1_y),1))
    rough2_y = np.resize(np.array(rough2_y),(len(rough2_y),1))
    plt.hist(rough0_y, bins=20, label='rough0')
    plt.hist(rough1_y, bins=10, label='rough1')
    plt.hist(rough2_y, bins=10, label='rough2')
    plt.legend()
    plt.xlabel('length')
    plt.show()
    
    xs = np.concatenate(rough0_x+rough1_x+rough2_x,axis=0)
    ys = np.concatenate((rough0_y,rough1_y,rough2_y), axis=0)
    vgg = Vgg16(vgg16_npy_path='./for_transfer_learning/vgg16.npy')
    print('Net built')
    for i in range(1000):
        b_idx = np.random.randint(0, len(xs), 50)
        train_loss = vgg.train(xs[b_idx], ys[b_idx])
        print(i, 'train loss: ', train_loss)

    vgg.save('./for_transfer_learning/model/transfer_learn')
    
def eval():
    vgg = Vgg16(vgg16_npy_path='./for_transfer_learning/vgg16.npy',
                restore_from='./for_transfer_learning/model/transfer_learn')
    vgg.predict(
        ['./rough1.0-1.5_node/6_right_6.jpg', './rough1.5-2.0/3_right_5.jpg','./rough1.0-1.5/14_right_6.jpg','./rough2.0-2.5/18_right_1.jpg'])
if __name__ == '__main__':
    #divide_img()
    #train()
    eval()

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Restoring parameters from ./for_transfer_learning/model/transfer_learn


/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


18_left_1.jpg
[[1.0843887]]
18_right_1.jpg
[[1.2929449]]
10_right_1.jpg
[[1.7972897]]
30_right_1.jpg
[[1.8826276]]
11_left_1.jpg
[[1.1074884]]
30_left_1.jpg
[[1.9716761]]
11_right_1.jpg
[[1.2535536]]
10_left_1.jpg
[[1.7777907]]
